# Project - Wrangle OpenStreetMap Data

Often the main part of a data scientists's role is to clean and wrangle the data into more accessible formats. While tedious and time-consuming, looking at the data at this early stages makes way for fewer inconsistencies later onwards.

Here, I have decide to use the OpenStreetMap data for my home city of New Delhi. 
New Delhi is the capital of India, a country in South Asia.

Note that the download is for a region known as NCR (National Capital Region), a set of places that includes New Delhi, and other adjoining places such as Purani Dilli (Old Delhi) and Gurgaon

As the name suggests, the OpenStreetMap project is an open-source project and comprises people tagging physical locations as XML tags, giving key details such as latitude and longitude. Other key details such as the type of the place ie amenity/hospital/highway is also specified, with its specific name as key-value pairs.

Consequently, the aim of this project is to explore the OpenStreetMap data, to clean the downloaded parts of the Delhi dataset, and find some features of the data.

There are parts to this data analysis:
1. Converting the XML data to CSV format
    1. Spotting errors
    2. Cleaning the data before conversion
    3. Converting the data into a CSV file
    4. Importing the CSV data into a MySQL database
      
2. Finding basic statistics about the data

3. Future improvements

In [3]:
# libraries that will be used 

# for SQL connections to the database
import MySQLdb

# for parsing the XML tree (library is built with C) and converting it into the CSV
import xml.etree.cElementTree as ET
import csv
import codecs

# validator library
import cerberus

#validation schema
import schema_osm

# dictionary module for simplifying life
from collections import defaultdict

# pretty printing
import pprint

# regular expressions
import re

# measuring time periods
import time

from itertools import islice

In [4]:
# osm file to convert
OSM_FILE = 'new-delhi_01.osm'

# 1. Converting the XML data to CSV format

Before we convert the data, let us take a look at some sample data:


In [5]:
def view_xml_tags(counter):
    '''
    view [counter] lines of xml_tags
    '''
    
    num_of_lines = 0

    for event, elem in ET.iterparse(OSM_FILE, events=("start",)):
        if (num_of_lines <= counter):
            num_of_lines += 1
            print elem.tag
            print elem.attrib

In [6]:
# view first 10 tags
view_xml_tags(10)

osm
{'timestamp': '2017-06-21T15:01:37Z', 'version': '0.6', 'generator': 'osmconvert 0.8.8'}
bounds
{'minlat': '28.183', 'maxlon': '77.733', 'minlon': '76.692', 'maxlat': '28.969'}
node
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:55Z', 'lon': '77.2159562', 'version': '53', 'user': 'saikabhi', 'lat': '28.6138967', 'id': '16173236'}
tag
{'k': 'admin_level', 'v': '2'}
tag
{'k': 'capital', 'v': 'yes'}
tag
{'k': 'is_capital', 'v': 'country'}
tag
{'k': 'is_in', 'v': 'National Capital Region, NCR, India'}
tag
{'k': 'is_in:continent', 'v': 'Asia'}
tag
{'k': 'is_in:country', 'v': 'India'}
tag
{'k': 'is_in:country_code', 'v': 'IN'}
tag
{'k': 'is_in:iso_3166_2', 'v': 'IN-DL'}


In [7]:
def sample(n, iterable):
    '''
    Return n items of the iterable as a list
    '''
    pprint.pprint (list(islice(iterable, n)))

Hopefully, the structure of the document is a bit more clear to you. Now, let's dive in!

#  1A. Cleaning the data before conversion

In order to spot issues with the data, we will consider a small segment of the data. This is because the actual file is 725 MB in size and doing any sort of basic calculations will be very time intensive!

For this analysis, I will be dealing with 10,000 tags as my small segment. 
While this may not look "small" to you at all, it is certainly MUCH smaller than the number of tags in 0.7 GB of data, and also a good number to be a representative sample of what the data contains.

The next cells list a few functions that I used to spot anomalies in the data. Their functionalities are listed in the doc strings below the function header.

In [8]:
NUM_TAGS = 10000

In [9]:
def get_keys(num_tags):
    '''
    Function that returns a dictionary of the first [num_tags] 'key' attributes of "tag" tags.
    '''
    
    count = 0
    dic_keys = {} 
    
    if (count <= num_tags):
        for event, elem in ET.iterparse(OSM_FILE, events=("start",)):
            count += 1
            if elem.tag == "tag":
                
                # check if in map
                if 'k' and 'v' in elem.attrib:
                    map_key = elem.attrib['k']
                    if map_key in dic_keys:
                        dic_keys[map_key] += 1
                    else:
                        dic_keys[map_key] = 1
    return dic_keys

In [10]:
dic_keys = get_keys(NUM_TAGS)

In [11]:
sample(10, dic_keys.iteritems())

# To see complete output, uncomment the next line
# pprint.pprint(dic_keys)

[('name:kg', 1),
 ('maxspeed', 1447),
 ('snowmobile', 14),
 ('IRrouterank', 29),
 ('is_in', 292),
 ('name:bat-smg', 2),
 ('name:kv', 1),
 ('created_by', 254),
 ('to', 9),
 ('source:tracer', 3)]


In [12]:
def get_values(num_tags):
    '''
    Function that returns a dictionary of the first [num_tags] 'value' attributes of "tag" tags.
    '''

    count = 0
    dic_values = {}

    if (count <= num_tags):
        for event, elem in ET.iterparse(OSM_FILE, events=("start",)):
            count += 1
            if elem.tag == "tag":
                
                # check if in map
                if 'k' and 'v' in elem.attrib:
                    map_key = elem.attrib['v']
                    if map_key in dic_values:
                        dic_values[map_key] += 1
                    else:
                        dic_values[map_key] = 1
                        
    return dic_values

In [15]:
dic_values = get_values(NUM_TAGS)

In [16]:
sample(10, dic_values.iteritems())

# To see complete output, uncomment the next line
# pprint.pprint(dic_values)

[('', 3),
 ('110005', 5),
 ('110006', 12),
 ('110007', 2),
 ('110001', 24),
 ('110002', 12),
 ('110003', 15),
 ('Patel Nagar Railway Station', 1),
 ('Tihar Jail', 2),
 ('110009', 13)]


In [17]:
def get_alpha_values(dic):
    '''
    Function that returns a dictionary of only the alphabetical keys of dictionary [dic].
    '''

    dic_alpha = defaultdict(set)
    
    for values in dic:
        values_lower = values.lower()
        if values_lower.isalpha(): 
            first_letter = values_lower[0]
            dic_alpha[first_letter].add(values)

    return dic_alpha

In [18]:
alpha_dic_values = get_alpha_values(dic_values)

In [25]:
# To see complete output, uncomment the next line
# pprint.pprint(alpha_dic_values)

In [36]:
def find_tag(attr_name, search_string, number_of_results = 5, number_of_searches = NUM_TAGS):
    '''
    Function that finds and prints tags whose attribute [attr_name] contains the string [search_string] as long 
    as the number of results is fewer than [number_of_results] and the number of tags searched is 
    fewer than [number_of_searches].
    
    The found tag attributes and its parent tag are printed for ease of access.
    '''
    
    results = 0
    searches = 0
    flag = 0
        
    for event, elem in ET.iterparse(OSM_FILE, events=("start",)):
        if elem.tag == "node":
            for children in elem:
                if results < number_of_results and searches < number_of_searches:

                # checking the conditions listed in the doc string
                    searches += 1

                    if children.attrib[attr_name].find(search_string) != -1 :

                        print children.attrib
                        print elem.attrib

                        results += 1
                else:
                    return

## Values that were problematic

### 1. Inconsistent time formats

In [37]:
time_keys = find_tag('k', 'hour')
time_values = find_tag('v', 'pm')

{'k': 'hour_on', 'v': '06:00'}
{'changeset': '37968208', 'uid': '17429', 'timestamp': '2016-03-21T06:03:04Z', 'lon': '77.0342984', 'version': '4', 'user': 'thevikas', 'lat': '28.4453755', 'id': '271373824'}
{'k': 'hour_off', 'v': '10:00'}
{'changeset': '37968208', 'uid': '17429', 'timestamp': '2016-03-21T06:03:04Z', 'lon': '77.0342984', 'version': '4', 'user': 'thevikas', 'lat': '28.4453755', 'id': '271373824'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2908758', 'uid': '26562', 'timestamp': '2009-10-21T05:35:56Z', 'lon': '77.0913044', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.489188', 'id': '537504262'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2908758', 'uid': '26562', 'timestamp': '2009-10-21T05:35:57Z', 'lon': '77.0911757', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.4891315', 'id': '537504263'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2915115', 'uid': '26562', 'timestamp': '2009-10-21T19:09:16Z', 'lon': '77.0860422', 'version': '1', '

If one looks at the time formats, you can see that they are all over the place: some have spaces between the A and M, some have lowercase, while others don't have this format altogether! There are also values of "24", which I take to be "24/7" by infering from nearby values. It is also very apparent that the values of the key of "opening_hours" contain all the required time values.

Thus, for standardization, I have chosen to use the 12-hour format (with AM and PM in uppercase and without the period between the individual letters), because I feel it is the easiest to read. The opening and closing times will be separated by a dash(-), for example: 10 AM - 4 PM.

### 2. Minor spelling errors

Because of the difference in spoken English and Hindi (the other major language spoken throughout New Delhi), there have been minor spelling errors such as "Dharampur" (a locality) and "Dharampura" with the extra "a". 

To find all such values, I wrote a crude string similarity function that finds such pairs of words. Keep in mind that finding similarity between strings is an active area of research, and that is is just a very basic function, so it might find certain words that are not as "similar" as we would want.

In [42]:
def string_sim(str1, str2):
    '''
    Crude similarity function for strings.
    Compares letter values at each string's positions and returns a ratio of similarity.
    If the ratio is greater than than a certain threshold, then the function outputs True, else False.
    '''
    
    lst1 = list(str1)
    lst2 = list(str2)
    
    lst1_no_spaces = []
    lst2_no_spaces = []
    
    problem_chars = [" ", "_"]
    
    for elem in lst1:
         if elem not in problem_chars:
            lst1_no_spaces.append(elem)
    
    for elem in lst2:
        if elem not in problem_chars:
            lst2_no_spaces.append(elem)
                
    smaller = min ( len(lst1_no_spaces), len(lst2_no_spaces) )
    
    if smaller == len(lst1_no_spaces):
        smaller_lst = lst1_no_spaces
        larger_lst = lst2_no_spaces
    else:
        smaller_lst = lst2_no_spaces
        larger_lst = lst1_no_spaces
        
    for i in range(len(larger_lst) - len(smaller_lst)):
        smaller_lst.append('X')
                
    count = 0
    
    if smaller > 3:
        for elem1, elem2 in zip(lst1_no_spaces, lst2_no_spaces):
        # print elem1, elem2
            if elem1 == elem2:
                count += 1
    
    return count/float(len(larger_lst)) >= 0.6

In [51]:
def find_similar(dic_values, num_results = 10, num_searches = NUM_TAGS):
    '''
    Function to iterate through dictionary values of [dic_values] to find all
    similar values and print them.
    '''

    searches = 0
    results = 0

    for values_1 in dic_values:
        for values_2 in dic_values:
            
            searches += 1
            # make sure the strings are alphabetical and start with the same letter: this ensures some smartness
            # in terms of the algorithms
            if (values_1.isalpha() and values_2.isalpha() and values_1 != values_2 and values_1[0] == values_2[0]):
                
                if (searches <= num_searches and results <= num_results):
                    
                    if string_sim(values_1, values_2):
                    
                        print "Value 1:", values_1
                        print "Value 2:", values_2
                        results += 1
                    
                else:
                    return

In [52]:
find_similar(dic_values)

As can be seen, even though my function is pretty mediocre, it has found quite a few values, for example:
1. "Bhuapur" and "Bhoapur"
2. "Narender" and "Nardendra"
3. "Electornics" and "Electronics"
4. "Titarpur" and "Tatarpur"
5. "Hindiston", "Hindıstan" and "Hindistan"
6. "Rampur" and "Rampura"

While there are many more, the problem with this approach is that it will have to be solved manually, and not programmatically, because one does not know *a priori* which values will be misspelt and what the misspelling will look like.

Because of the above problem, I dug deeper to find a simpler connection between the spellings. I noticed that the anglacized names of places in Hindi add an extra 'a' for better pronunciation, and so I set out to find such pairs of words. 

The advantage of narrowing down the differences in the words is that this particular problem can be fixed programmatically. 

In [17]:
extra_chars = ['a']

for values_1 in dic_values:
    for values_2 in dic_values:
            
        for endings in extra_chars:
            if values_2 == values_1 + endings:

                print "Value 1:", values_1
                print "Value 2:", values_2

Value 1: Sunpur
Value 2: Sunpura
Value 1: Susheel
Value 2: Susheela
Value 1: Nathupur
Value 2: Nathupura
Value 1: Dharampur
Value 2: Dharampura
Value 1: Rampur
Value 2: Rampura


Thus, we can look to fix the places differing by an extra 'a'.

### 3. Misformated strings: example - DTC

The Delhi Transport Corporation is one of Delhi's biggest bus operators. 

While going through the XML file, I found that the DTC was written both as its acronym (DTC) and its full version (Delhi Transport Corporation). Because this is a globally accessible dataset, I decided to keep the full version (which includes the acronym) so that this makes more sense to people who decide to investigate it.

In [39]:
find_tag('v', 'DTC')

{'k': 'operator', 'v': 'DTC'}
{'changeset': '30470980', 'uid': '1751211', 'timestamp': '2015-04-25T10:14:28Z', 'lon': '77.1656102', 'version': '2', 'user': 'ajantriks', 'lat': '28.5461287', 'id': '566993158'}
{'k': 'operator', 'v': 'DTC'}
{'changeset': '30467334', 'uid': '1751211', 'timestamp': '2015-04-25T06:14:21Z', 'lon': '77.1682489', 'version': '2', 'user': 'ajantriks', 'lat': '28.5493627', 'id': '566993164'}
{'k': 'name', 'v': 'Kashmiri Gate DTC Terminus'}
{'changeset': '5961686', 'uid': '1306', 'timestamp': '2010-10-05T14:25:22Z', 'lon': '77.2283796', 'version': '1', 'user': 'PlaneMad', 'lat': '28.6685915', 'id': '938534499'}
{'k': 'name', 'v': 'Sector 10 DTC bust stop'}
{'changeset': '6400961', 'uid': '372318', 'timestamp': '2010-11-18T15:56:23Z', 'lon': '77.0565582', 'version': '1', 'user': 'rkumar02', 'lat': '28.5898321', 'id': '993723518'}
{'k': 'name', 'v': 'DTC Depot'}
{'changeset': '13779743', 'uid': '56597', 'timestamp': '2012-11-06T22:20:37Z', 'lon': '77.0536523', 'vers

## Values that seemed problematic, but were alright

In [40]:
find_tag('v', 'AND')

{'k': 'source', 'v': 'AND'}
{'changeset': '12138166', 'uid': '56597', 'timestamp': '2012-07-07T09:21:32Z', 'lon': '76.7811767', 'version': '3', 'user': 'Oberaffe', 'lat': '28.2048836', 'id': '245764539'}
{'k': 'source', 'v': 'AND'}
{'changeset': '15964016', 'uid': '91490', 'timestamp': '2013-05-03T18:37:27Z', 'lon': '76.7534322', 'version': '3', 'user': 'Heinz_V', 'lat': '28.2054048', 'id': '245764549'}
{'k': 'source', 'v': 'AND'}
{'changeset': '12067458', 'uid': '56597', 'timestamp': '2012-06-30T16:19:37Z', 'lon': '77.1076424', 'version': '3', 'user': 'Oberaffe', 'lat': '28.23457', 'id': '245764666'}
{'k': 'source', 'v': 'AND'}
{'changeset': '37945233', 'uid': '439726', 'timestamp': '2016-03-19T20:37:11Z', 'lon': '77.6588646', 'version': '2', 'user': 'chandusekharreddy', 'lat': '28.2584839', 'id': '245764757'}
{'k': 'source', 'v': 'AND'}
{'changeset': '3360495', 'uid': '17429', 'timestamp': '2009-12-13T05:44:23Z', 'lon': '77.1239532', 'version': '3', 'user': 'thevikas', 'lat': '28.265

While glancing through the XML document, I found a lot of key-value pairs of "source" and "AND". 
Bewildered, I was sure I had spotted a problem, because "AND" did not seem correct at all to me.

However, some searching on the <a href = "http://wiki.openstreetmap.org/wiki/Key:source">OSM Wiki</a> revealed otherwise.

Under common "source" values, I found a <a href = "http://wiki.openstreetmap.org/wiki/AND_Data">link</a> to the "AND" value, which states, "On the 4th of July 2007 AND Automotive Navigation Data donated the entire streetmap of the Netherlands as well as road networks for China and India... We also have the basic India road network and city names AND data in place...".

AHA! This makes complete sense now! The source of the data was through the AND dataset! This is infact important meta-data that should in fact be retained. 

"Potlatch 0.9a" is another software used that has been used as a source.

In [41]:
find_tag('v', 'JOSM')

{'k': 'created_by', 'v': 'JOSM'}
{'changeset': '19305', 'uid': '17429', 'timestamp': '2008-12-01T16:19:34Z', 'lon': '77.0576899', 'version': '9', 'user': 'thevikas', 'lat': '28.4690552', 'id': '249077809'}
{'k': 'created_by', 'v': 'JOSM'}
{'changeset': '19305', 'uid': '17429', 'timestamp': '2008-12-01T16:19:35Z', 'lon': '77.0703931', 'version': '7', 'user': 'thevikas', 'lat': '28.4792124', 'id': '249077810'}
{'k': 'created_by', 'v': 'JOSM'}
{'changeset': '508147', 'uid': '17429', 'timestamp': '2008-05-24T09:07:39Z', 'lon': '77.0701475', 'version': '4', 'user': 'thevikas', 'lat': '28.4592792', 'id': '249132367'}
{'k': 'created_by', 'v': 'JOSM'}
{'changeset': '500264', 'uid': '17429', 'timestamp': '2008-05-22T19:50:30Z', 'lon': '77.0583362', 'version': '3', 'user': 'thevikas', 'lat': '28.4680855', 'id': '249132371'}
{'k': 'created_by', 'v': 'JOSM'}
{'changeset': '1779803', 'uid': '138012', 'timestamp': '2009-07-09T11:34:43Z', 'lon': '77.0658862', 'version': '3', 'user': 'thepatel', 'lat'

The "JOSM" value was another one that confused me. With a key - value of "created_by": "JOSM", I was pretty sure that this was redundant information, since the parent "node" tag for each "tag" tag already contained the user who contributed that point. 

dispHowever, once I Googled this term, I found to my surprise that JOSM in fact an editor for OSM Maps for <a href = "https://josm.openstreetmap.de/">J</a>ava. As this is important information about the points, it should be retained.

In [69]:
def display_tags(num = 20):

    num_results = 0

    for event, elem in ET.iterparse(OSM_FILE, events=("start",)):
        if (count <= NUM_TAGS):
            if elem.tag == "node":
                for children in elem:
                    if num_results <= num:
                        if len(children.attrib[attr]) == 2 or not children.attrib[attr].isalnum():
                            print children.attrib
                            num_results += 1

                    else:
                        return

In [70]:
display_tags()

{'k': 'admin_level', 'v': '2'}
{'k': 'is_capital', 'v': 'country'}
{'k': 'is_in', 'v': 'National Capital Region, NCR, India'}
{'k': 'is_in:continent', 'v': 'Asia'}
{'k': 'is_in:country', 'v': 'India'}
{'k': 'is_in:country_code', 'v': 'IN'}
{'k': 'is_in:iso_3166_2', 'v': 'IN-DL'}
{'k': 'name:ace', 'v': 'New delhi'}
{'k': 'name:af', 'v': 'Nieu-Delhi'}
{'k': 'name:am', 'v': u'\u1292\u12cd \u12f4\u120a'}
{'k': 'name:an', 'v': 'Nueva Delhi'}
{'k': 'name:ang', 'v': u'N\u012b\u01bfe Delhi'}
{'k': 'name:ar', 'v': u'\u0646\u064a\u0648\u062f\u0644\u0647\u064a'}
{'k': 'name:bat-smg', 'v': u'Naujas\u0117s Del\u0117s'}
{'k': 'name:be', 'v': u'\u041d\u044c\u044e-\u0414\u044d\u043b\u0456'}
{'k': 'name:be-tarask', 'v': u'\u041d\u044c\u044e-\u0414\u044d\u043b\u0456'}
{'k': 'name:bg', 'v': u'\u041d\u044e \u0414\u0435\u043b\u0445\u0438'}
{'k': 'name:bn', 'v': u'\u09a8\u09a4\u09c1\u09a8 \u09a6\u09bf\u09b2\u09cd\u09b2\u09bf'}
{'k': 'name:bo', 'v': u'\u0f53\u0f7a\u0f60\u0f74\u0f0b\u0f51\u0f72\u0f63\u0f0b\u0

I was trying to find problematic values: in general, shorter and non-alpha numeric values are more likely to be problematic. 

If you look above at the data, you will see most of these "problematic" values are unicode characters (\u values).
However, you will see that these are in fact names of India in different languages, as can be seen in the key-value pairs: "name:XX" : "\u1292\u12cd \u12f4\u120a". 

Because India is a diverse country with more than 600 regional languages, this kind of <b>linguistic difference</b> makes sense.

Another language related problem was the spellings of "India" such as "Indya" and "Indija". 
Once again, this is not an error, and simply a translation of the regional spellings of India into English.

### B. Cleaning the data before conversion
Now that we have identified quite a few problems with the data, let us tackle them one by one!

### 1. Inconsistent time formats

In [54]:
find_tag('k', 'opening_hours')

{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2908758', 'uid': '26562', 'timestamp': '2009-10-21T05:35:56Z', 'lon': '77.0913044', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.489188', 'id': '537504262'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2908758', 'uid': '26562', 'timestamp': '2009-10-21T05:35:57Z', 'lon': '77.0911757', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.4891315', 'id': '537504263'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2915115', 'uid': '26562', 'timestamp': '2009-10-21T19:09:16Z', 'lon': '77.0860422', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.4809232', 'id': '537994220'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2915115', 'uid': '26562', 'timestamp': '2009-10-21T19:09:16Z', 'lon': '77.0860364', 'version': '1', 'user': 'Nishant Sharma', 'lat': '28.4810242', 'id': '537994221'}
{'k': 'opening_hours', 'v': '24/7'}
{'changeset': '2915115', 'uid': '26562', 'timestamp': '2009-10-21T19:09:16Z', 'lon': '77.0860307'

In [55]:
# mappings from incorrect spellings to correct spellings 
# (based on sample output from above)

update_times = {
        '24': "'24/7", # extra apostrophe to prevent conversion of 24/7 to 24 July
        'mo': 'Monday',
        'mon': 'Monday',
        'tu': 'Tuesday',
        'we': 'Wednesday',
        'wed' 'Wednesday'
        'th': 'Thursday',
        'thur': 'Thursday',
        'fr': 'Friday',
        'sa': 'Saturday',
        'sat': 'Saturday',
        'su': 'Sunday',
        'sun': 'Sunday',
        'a.m': 'AM',
        'a.m.': 'AM',
        'am': 'AM',
        'p.m': 'PM',
        'p.m.': 'PM',
        'pm': 'PM',
        'to': '-'
    }

In [56]:
def cleanup_times(word):
    '''
    Function to cleanup the time format as per standards listed in above cells
    '''    
    try:
    
    # separate characters such as '10am' to obtain '10 am' for parsing
        needs_spacing = re.compile('[0-9][a-z]')
        space_pos = needs_spacing.findall(word)

        for phrases in space_pos:
            word_pos = word.find(phrases)
            word = word[:word_pos + 1] + " " + word[word_pos + 1:]

        # find all instances of 24 hour time and convert them
        time_format = re.compile('[0-9]+:[0-9]+')
        time_lst = time_format.findall(word)

        for times in time_lst:

            start = word.find(times)
            length = len(times)
            end = start + length

            colon = times.find(":")
            hour_int = int(times[:colon])
            minute_str = times[colon:end]

            if hour_int >= 12:
                if hour_int != 12:
                    hour_int -= 12
                time_str = str(hour_int)
                time_str += minute_str
            else:
                time_str = str(hour_int)
                time_str += minute_str    

            word = word[:start] + time_str + word[end:] 

        # separate characters such as '-' from individual words for better parsing
        new_word = ""

        for letter_pos in range(len(word)):

            if word[letter_pos] in ["-", ","]:
                if word[letter_pos - 1] != " " and word[letter_pos + 1] == " ":
                    new_word += " " + word[letter_pos]

                elif word[letter_pos - 1] == " " and word[letter_pos + 1] != " ":
                    new_word += word[letter_pos] + " "

                elif word[letter_pos - 1] != " " and word[letter_pos + 1] != " ":
                    new_word += " " + word[letter_pos] + " "

            else:
                new_word += word[letter_pos]

        # update words as per dictionary values
        sentence_lst = new_word.split()

        answer_lst = []


        for words in sentence_lst:

            # lower
            words = words.lower()

            # check if in dict
            if words in update_times:
                answer_lst.append(update_times[words])
            else:
                answer_lst.append(words)

        extra_spaces_word =  " ".join(answer_lst)

        final_word = ""

        # remove extra spaces fromm commas
        for letter_pos in range(len(extra_spaces_word)):
            if extra_spaces_word[letter_pos] == " " and extra_spaces_word[letter_pos + 1] == ",":
                pass
            elif extra_spaces_word[letter_pos] == "," and extra_spaces_word[letter_pos + 1] != " ":
                final_word += ", "

            else:
                final_word += extra_spaces_word[letter_pos]

        return final_word
    
    except:
        return word

In [68]:
# example use of function
cleanup_times('10a.m. to 2pm')

'10 AM - 2 PM'

### 2. Minor spelling errors

In [69]:
def fix_extra_a(str1, set_of_values):
    '''
    if [str1] has an extra "a", and its correct spellings has
    been encountered, "fix" it
    '''
    try:
        set_of_values.add(str1)
    
        if str1[-1] == "a" and (str1[:-1] in set_of_values):
            return str1[:-1]
    except:
        pass
    return str1

### 3. Misformatted string - example: DTC

In [70]:
def format_string_dtc(value):
    return "Delhi Transport Corporation ( DTC )"

### C. Converting the data into a CSV file

Now that we have successfully dealt with a few issues programmatically, and are aware of other factors, we are ready to convert!

The function below converts AND validates the conversion (based on the schema in schema_osm.py). This ensures that the conversion that takes place is correct. 

Conversion is done on the ENTIRE file and may take longer.

In [71]:
# taken from Udacity course quiz and edited afterwards
# note that cells in Excel may have weird formatting, but rest assured, they retain their values

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files


## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

"""

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"

SCHEMA = schema_osm.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

NODE_ATTRIB = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAG_ATTRIB = ['k', 'v']
WAY_ATTRIB = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SET_OF_VALUES = set()

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    elem = element
    
    # print elem.tag
    if elem.tag == "node":
        node_attribs = {}
        for attributes in NODE_ATTRIB:
            node_attribs[attributes] = elem.attrib[attributes]
                
            # children "tag" elements
            tags = []
            
        # deleting extraneous time info
        extra_pos = node_attribs['timestamp'].find('T')
        node_attribs['timestamp'] = node_attribs['timestamp'][:extra_pos]
            
    count = 0
    for child in elem:
        
        if child.tag == "tag":    
            node_tag_attrib_dict = {}

            for attributes in NODE_TAG_ATTRIB:
                
                node_tag_attrib_dict[attributes] = child.attrib[attributes]   
                node_tag_attrib_dict['id'] = elem.attrib['id']

                # filter
                colon_pos = child.attrib["k"].find(":") 
                # problematic

                if colon_pos == -1:
                    node_tag_attrib_dict['type'] = "regular"
                else:
                    node_tag_attrib_dict['type'] = child.attrib["k"][:colon_pos]
                    node_tag_attrib_dict['k'] = child.attrib["k"][colon_pos + 1:]

            node_tag_attrib_dict['key'] = node_tag_attrib_dict['k']
            del node_tag_attrib_dict['k']

            node_tag_attrib_dict['value'] = node_tag_attrib_dict['v']
            del node_tag_attrib_dict['v']
               
            if node_tag_attrib_dict['key'] == "opening_hours":
                node_tag_attrib_dict['value'] = cleanup_times(node_tag_attrib_dict['value'])
                
            node_tag_attrib_dict['value'] = fix_extra_a(node_tag_attrib_dict['value'], SET_OF_VALUES)    
            
            if node_tag_attrib_dict['key'] == "DTC":
                format_string_dtc(node_tag_attrib_dict['value'])

            # add to list
            tags.append(node_tag_attrib_dict)
        
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file:
#          codecs.open(WAYS_PATH, 'w') as ways_file, \
#          codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
#          codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
#         ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
#         way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
#         way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
#         ways_writer.writeheader()
#         way_nodes_writer.writeheader()
#         way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])

process_map(OSM_FILE, validate=True)


### 4. Importing the CSV data into a MySQL database

Importing the CSV into MySQL is easy; we just need to ensure that there is a connection to the database and all tables, in which we want to put data, exist.

Putting the data will take time.

In [86]:
# Open database connection
# Put your database credentials here
USER = "root"
PWORD = "root"

db = MySQLdb.connect("localhost", USER, PWORD)

# prepare a cursor object using cursor() method
cursor = db.cursor()

# drop any table with the same name and create a new one
sql = 'DROP DATABASE IF EXISTS osm'
cursor.execute(sql)

sql = 'CREATE DATABASE IF NOT EXISTS osm'
cursor.execute(sql)

1L

In [87]:
sql1 = '''CREATE TABLE IF NOT EXISTS nodes ( 
    id BIGINT PRIMARY KEY NOT NULL, 
    lat REAL, 
    lon REAL, 
    user TEXT, 
    uid BIGINT, 
    version BIGINT, 
    changeset BIGINT, 
    timestamp DATE 
    );'''

# Because key is a keyword, it has to be enclosed in `key`
sql2 = '''CREATE TABLE IF NOT EXISTS nodes_tags ( 
    id BIGINT,
    `key` TEXT, 
    `value` TEXT, 
    type TEXT, 
    FOREIGN KEY (id) REFERENCES nodes(id) 
    );'''

In [88]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

sql_list = [sql1, sql2]
for sqls in sql_list:
    cursor.execute(sqls)
    
# close database
db.close()

In [89]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

# counters for accepted and rejected tags
count_good = 0
count_bad = 0

csv_data = csv.reader(file('nodes.csv') )
for row in csv_data:
    
    # prevent header from getting stored into the database
    if row[0] == 'id':
        continue
        
    # in case any error occurs
    try:
        iid = row[0]
        lat = row[1]
        lon = row[2]
        user = row[3]
        uid = row[4]
        version = row[5]
        changeset = row[6]
        timestamp = row[7]

        execution = 'INSERT INTO `nodes` (`id`, `lat`, `lon`, `user`, `uid`, `version`, `changeset`, `timestamp`) VALUES (' + iid + ', ' + lat + ', ' + lon + ', ' + '"' + user + '"' + ', ' + uid + ', ' + version + ', ' + changeset + ', ' + '"' + timestamp + '"' + ')'
    
        # print execution

        cursor.execute(execution)
        count_good += 1
    
    
    except:
        
        # print bad input
        print execution
        count_bad += 1
        

db.commit()

# close the connection to the database.
cursor.close()

print "Accepted: " + str(count_good)
print "Rejected: " + str(count_bad)
print "Accuracy: " + str(round(float(count_good)/(count_bad + count_good), 4))

Accepted: 3406392
Rejected: 0
Accuracy: 1.0


It turns out that out of the approximately 340 thousand tags, all made it to the database! Good data cleaning!


In [90]:
# Definitions of variable similar to above

db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

count_good = 0
count_bad = 0

csv_data = csv.reader(file('nodes_tags.csv') )
for row in csv_data:
    if row[0] == 'id':
        continue
    try:
        iid = row[0]
        key = row[1]
        value = row[2]
        types = row[3]

        execution = 'INSERT INTO `nodes_tags` (`id`, `key`, `value`, `type`) VALUES ' + '(' + iid + ', ' + '"' + key + '"' + ', ' + '"' + value + '"' + ', ' + '"' + types + '"'+ ')'
        # print execution

        cursor.execute(execution)
        db.commit()
        count_good += 1

    except:
        print execution
        count_bad += 1
        
    
# close the connection to the database.
cursor.close()

print "Accepted: " + str(count_good)
print "Rejected: " + str(count_bad)
print "Accuracy: " + str(round(float(count_good)/(count_bad + count_good) , 4))

INSERT INTO `nodes_tags` (`id`, `key`, `value`, `type`) VALUES (539314057, "housenumber", "140\", "addr")
INSERT INTO `nodes_tags` (`id`, `key`, `value`, `type`) VALUES (3832251782, "fixme", "Yes bank is not in that corner. I found Yes bank near "Centre for Policy Research", more exactly: 28.601838 77.189462", "regular")
Accepted: 41401
Rejected: 2
Accuracy: 1.0


The reason behind the rejected tags:
1. the first has a special character \ in the "140\ " that escapes the \
2. the second uses double quotes within a field, where double quotes are also used for delimiting certain fields.

Even so, the number of errors is not huge, and comparable to what one can find in such datasets.

### 2. Finding basic statistics about the data

Size of the file:
1. 24.6 MB (compressed)
2. 725 MB (uncompressed)

### Number of `way` tags

This are none; hence, they are not even searched for in the XML.

### Number of `node` tags

3406392

In [91]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT COUNT(*) FROM nodes'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
print result
    
cursor.close()

((3406392L,),)


### Number of `tag` tags

41401


In [92]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT COUNT(*) FROM nodes_tags'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
print result

cursor.close()

((41401L,),)


### Number of distinct users

1311


In [93]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT COUNT(DISTINCT user) FROM nodes'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
print result
    
cursor.close()

((1311L,),)


### Most active users

Oberaffe with 223946 is the most active

In [94]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT user, COUNT(*) AS count FROM nodes GROUP BY user ORDER BY count DESC LIMIT 5'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('Oberaffe', 223946L)
('premkumar', 128796L)
('saikumar', 127139L)
('Naresh08', 121068L)
('anushap', 110283L)


### Most active time

3 June 2015, with 163392 edits!

In [95]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT timestamp, COUNT(*) AS count FROM nodes GROUP BY timestamp ORDER BY count DESC LIMIT 10'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

(datetime.date(2015, 6, 3), 163392L)
(datetime.date(2015, 6, 8), 158008L)
(datetime.date(2015, 6, 5), 156841L)
(datetime.date(2015, 6, 9), 151059L)
(datetime.date(2015, 6, 4), 147670L)
(datetime.date(2015, 5, 25), 145605L)
(datetime.date(2015, 6, 2), 142810L)
(datetime.date(2015, 6, 6), 135451L)
(datetime.date(2015, 6, 1), 117400L)
(datetime.date(2016, 5, 4), 102349L)


### Number of address edits

4578

In [96]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT type, COUNT(*) AS count FROM nodes_tags GROUP BY type HAVING type = "addr"'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('addr', 4578L)


### Types of `tag` edits

tower: 16072

"towers" here refers to towers on the electricity grid.

IN: 5th -> Makes sense as "IN" refers to India

Gurgaon: top 10 -> This is an adjoining area of New Delhi; this again makes sense as the map data contains information not only about New Delhi, but the National Capital Region.

Similarly, '110006' (~ 30th) is the **postal code** for Chandni Chowk (not coincendentally the value below it on the list), a famous market in Old Delhi.

In [58]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT value, COUNT(*) AS count FROM nodes_tags GROUP BY value ORDER BY count DESC LIMIT 10'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('tower', 16072L)
('locality', 890L)
('yes', 764L)
('traffic_signals', 557L)
('tree', 538L)
('IN', 379L)
('station', 347L)
('street_vendor', 329L)
('gate', 327L)
('bus_stop', 279L)


### Most edits to Chandni Chowk (cross table query)

From my above findings, I was curious to know who was adding all these locations in Chandni Chowk on the OSM dataset, so I queried for both the postal code and name (queries taken *verbatim* from the result list above)

It looks like 'thevikas' (probably someone called Vikas) knows a lot about Old Delhi!

In [3]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT user, COUNT(*) AS count FROM nodes WHERE id in \
    (SELECT id FROM nodes_tags WHERE value = "110006" OR value = "Lajpat Rai Market, Chandni Chowk, Delhi") GROUP BY user ORDER BY count DESC'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('thevikas', 56L)
('Oberaffe', 3L)
('Jitin Kumar', 2L)
('Claudio Acosta', 1L)
('Usma Dhammei', 1L)
('yoyi79', 1L)


### Distribution of types of amenities

As was expected, *frequently visited* amenities such as restaurants, ATMs, places of worship and banks are the most common.

In [59]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT `value`, COUNT(*) AS count FROM nodes_tags WHERE `key` =  "amenity" GROUP BY `value` ORDER BY count DESC LIMIT 15'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('restaurant', 223L)
('fuel', 213L)
('atm', 202L)
('place_of_worship', 175L)
('bank', 169L)
('school', 158L)
('fast_food', 131L)
('parking', 89L)
('hospital', 88L)
('cafe', 78L)
('pharmacy', 71L)
('police', 52L)
('cinema', 43L)
('embassy', 42L)
('toilets', 41L)


### Distribution of cafes

78 mentions of "cafe"

In [31]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT `value`, COUNT(*) AS count FROM nodes_tags WHERE `key` =  "amenity" AND `value` = "cafe" GROUP BY `value`'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('cafe', 78L)


### Number of education centres

As can be seen, New Delhi seems to be an education hub, with 8 kindergartens, 26 colleges and a whopping 158 schools!

In [38]:
db = MySQLdb.connect("localhost", "root", "root", "osm")
cursor = db.cursor()

execution = 'SELECT `value`, COUNT(*) AS count FROM nodes_tags WHERE `key` =  "amenity" AND (`value` = "college" OR `value`  = "school" OR `value` = "kindergarten") GROUP BY `value`'
cursor.execute(execution)
db.commit()

result = cursor.fetchall()
for rows in result:
    print rows

cursor.close()

('college', 26L)
('kindergarten', 8L)
('school', 158L)


## 3. Future improvements

Phew! We are finally done!

However much we may have accomplished, there is still more to be done!

Here are certain suggestions:
1. 'is_in:XXX' occurs a lot as a key in key-value pairs. This strikes me as odd because the words after 'is_in' are self-explanatory, making 'is_in' redundant. 
2. Checking for common spelling errors: in my analysis, I come upon a lot of misspelt words such as "SMOOTHNES". 

In [60]:
find_tag('k', 'is_in')

{'k': 'is_in', 'v': 'National Capital Region, NCR, India'}
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:55Z', 'lon': '77.2159562', 'version': '53', 'user': 'saikabhi', 'lat': '28.6138967', 'id': '16173236'}
{'k': 'is_in:continent', 'v': 'Asia'}
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:55Z', 'lon': '77.2159562', 'version': '53', 'user': 'saikabhi', 'lat': '28.6138967', 'id': '16173236'}
{'k': 'is_in:country', 'v': 'India'}
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:55Z', 'lon': '77.2159562', 'version': '53', 'user': 'saikabhi', 'lat': '28.6138967', 'id': '16173236'}
{'k': 'is_in:country_code', 'v': 'IN'}
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:55Z', 'lon': '77.2159562', 'version': '53', 'user': 'saikabhi', 'lat': '28.6138967', 'id': '16173236'}
{'k': 'is_in:iso_3166_2', 'v': 'IN-DL'}
{'changeset': '46299189', 'uid': '3029661', 'timestamp': '2017-02-22T08:42:

Benefits and Anticipated Problems 1 are in regard to point 1 made above.

Benefits and Anticipated Problems 2 are in regard to point 2 made above.

### Benefits:
1. This can be easily cleaned programmatically; however, the problem is that 'is_in' is ONE example of redundant tags. 
2. Easy to clean a certain subset of spelling mistakes programmatically! - see above

### Anticipated Problems:
1. Finding any more redundant tags still requires a manual sweep at first! :(
2. However, as much as I caught certain typical errors, I am unable to catch errors for each and every word. Furthermore, in the current scope of the project, it is impossible to predict the next word in order to suggest improvements.

Both reasons above point to the use of machine learning, artificial intelligence and Natural Language Processing (NLP). With those tools in our arsenal, we could do much more sophisticated analysis and cleaning. Let me elaborate on what I said: NLP and ML in combination could be used to predict the most common misspellings of words. These could further act as training data for the model, and over time it would become better. Artificial intelligence could be used to smartly guess common patterns of dirty data. Thus, this trio of skills could be really very useful!
   
Another way to improve this dataset would be to gamify this experience: the more edits you make, the higher score you have. A feature that could be added to this is extra points for editing the place at which you're currently standing (making use of the GPS)!. These could seriously improve participation and improve this project to a large extent. 
    
A last point would be to use an API like Google Maps. I did try this; however, because the API autocorrects incorrect spellings, it is of little use in determining if a place really exists.